In [1]:
import cv2
import subprocess
import os

In [2]:
def get_cam_index(n_cams=1):
    """
    Gets the index of USB cameras connected to the system.

    Parameters:
    - n_cams (int): Number of camera indices to retrieve. Defaults to 1.

    Returns:
    - int or tuple: If n_cams is 1, returns the index of the first camera found.
                    If n_cams is 2, returns a tuple with indices of the first two cameras found.

    Raises:
    - ValueError: If n_cams is not 1 or 2.
    - RuntimeError: If the command 'v4l2-ctl --list-devices' fails to execute or if v4l-utils is not installed.

    How it works: 
    The function executes the command 'v4l2-ctl --list-devices' using subprocess.
    This command lists the available video devices and their associated paths (/dev/videoX).
    It parses the output to extract the indices of the video devices.
    
    Example output of 'v4l2-ctl --list-devices':
    
    USB Camera: USB Camera (usb-0000:00:14.0-3.3):
        /dev/video2
        /dev/video3
        /dev/media1

    USB2.0 HD UVC WebCam: USB2.0 HD (usb-0000:00:14.0-7):
        /dev/video0
        /dev/video1
        /dev/media0

    The index of each camera is the number after '/dev/video'.
    The function returns the index or indices based on the value of n_cams.
     
    Note:
    - This function assumes that v4l-utils package (which includes v4l2-ctl) is installed locally.
    - If v4l-utils is not installed, install it by running 'sudo apt-get install v4l-utils' from the linux terminal.
    """
    
    try:
        result = subprocess.run(['v4l2-ctl', '--list-devices'], capture_output=True, text=True, check=True)
        out = result.stdout.splitlines()
    except subprocess.CalledProcessError as e:
        raise RuntimeError("Failed to execute v4l2-ctl. Make sure v4l-utils is installed. Install it by running 'sudo apt-get install v4l-utils' from the linux terminal.") from e

    indices = []
    for line in out:
        if "/dev/video" in line:
            indices.append(int(line.strip()[-1]))
    
    if n_cams == 1:
        if len(indices) < 4:
            raise ValueError("No external cameras found.")
        return indices[0]
    elif n_cams == 2:
        if len(indices) < 6:
            raise ValueError("Less than 2 external cameras found.")
        return indices[0], indices[2]
    else:
        raise ValueError(f"Value of n_cams = {n_cams} is out of bounds. It must be either 1 or 2.")


def click_an_image(n_cams=1, brightness=50, contrast=60, saturation=70, hue=0, temperature=4600, exposure=1000, pan=0):
    """
    Captures an image from USB cameras and displays them.

    Parameters:
    - n_cams (int): Number of cameras to capture from. Defaults to 1.
    - brightness (int): Brightness level (0-100). Defaults to 50.
    - contrast (int): Contrast level (0-100). Defaults to 60.
    - saturation (int): Saturation level (0-100). Defaults to 70.
    - hue (int): Hue level (0-100). Defaults to 0.
    - temperature (int): White balance temperature (Kelvin). Defaults to 4600.
    - exposure (int): Exposure time in milliseconds. Defaults to 1000.
    - pan (int) : min=-57600 max=57600 step=3600 default=0 value=0.

    Raises:
    - ValueError: If n_cams is not 1 or 2.

    Notes:
    - Uses OpenCV (cv2) to interact with the cameras and capture images.
    - Requires v4l-utils package installed to function properly.
    """

    if n_cams == 1:
        cap_index_1 = get_cam_index(n_cams)
        # Open a connection to the camera.
        cap = cv2.VideoCapture(cap_index_1)

        # Check if camera opened successfully
        if not cap.isOpened():
            print("Error: Could not open camera.")
            return

        # Set camera properties
        cap.set(cv2.CAP_PROP_BRIGHTNESS, brightness)
        cap.set(cv2.CAP_PROP_CONTRAST, contrast)
        cap.set(cv2.CAP_PROP_SATURATION, saturation)
        cap.set(cv2.CAP_PROP_HUE, hue)
        cap.set(cv2.CAP_PROP_TEMPERATURE, temperature)
        cap.set(cv2.CAP_PROP_EXPOSURE, exposure)
        cap.set(cv2.CAP_PROP_PAN, pan)

        # Print current camera properties
        print("Brightness:", cap.get(cv2.CAP_PROP_BRIGHTNESS))
        print("Contrast:", cap.get(cv2.CAP_PROP_CONTRAST))
        print("Saturation:", cap.get(cv2.CAP_PROP_SATURATION))
        print("Hue:", cap.get(cv2.CAP_PROP_HUE))
        print("White Balance Temperature:", cap.get(cv2.CAP_PROP_TEMPERATURE))
        print("Exposure:", cap.get(cv2.CAP_PROP_EXPOSURE))

        # Capture frame-by-frame
        ret, frame = cap.read()

        # Check if the frame was captured successfully
        if not ret:
            print("Error: Could not read frame.")
            return

        # Display the captured frame
        cv2.imshow('Camera Capture', frame)

        # Save the captured frame to a file
        cv2.imwrite('captured_image.jpg', frame)

        # Wait for a key press and close all OpenCV windows
        cv2.waitKey(0)
        cv2.destroyAllWindows()

        # Release the camera
        cap.release()

    elif n_cams == 2:
        cap_index_1, cap_index_2 = get_cam_index(n_cams)
        
        # Open connections to both cameras
        cap1 = cv2.VideoCapture(cap_index_1)
        cap2 = cv2.VideoCapture(cap_index_2)

        # Check if cameras opened successfully
        if not cap1.isOpened() or not cap2.isOpened():
            print("Error: Could not open one or more cameras.")
            return

        # Set camera properties for both cameras
        cap1.set(cv2.CAP_PROP_BRIGHTNESS, brightness)
        cap1.set(cv2.CAP_PROP_CONTRAST, contrast)
        cap1.set(cv2.CAP_PROP_SATURATION, saturation)
        cap1.set(cv2.CAP_PROP_HUE, hue)
        cap1.set(cv2.CAP_PROP_TEMPERATURE, temperature)
        cap1.set(cv2.CAP_PROP_EXPOSURE, exposure)
        cap1.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap1.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        cap1.set(cv2.CAP_PROP_PAN, pan)
        
        cap2.set(cv2.CAP_PROP_BRIGHTNESS, brightness)
        cap2.set(cv2.CAP_PROP_CONTRAST, contrast)
        cap2.set(cv2.CAP_PROP_SATURATION, saturation)
        cap2.set(cv2.CAP_PROP_HUE, hue)
        cap2.set(cv2.CAP_PROP_TEMPERATURE, temperature)
        cap2.set(cv2.CAP_PROP_EXPOSURE, exposure)
        cap2.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap2.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        cap2.set(cv2.CAP_PROP_PAN, pan)

        # Capture frames from both cameras
        ret1, frame1 = cap1.read()
        ret2, frame2 = cap2.read()

        # Check if frames were captured successfully
        if not ret1 or not ret2:
            print("Error: Could not read frames from one or more cameras.")
            return

        # Display frames from both cameras
        cv2.imshow('Camera 1 Capture', frame1)
        cv2.imshow('Camera 2 Capture', frame2)

        # Save the captured frames to files
        cv2.imwrite('captured_image_cam1.jpg', frame1)
        cv2.imwrite('captured_image_cam2.jpg', frame2)

        # Wait for a key press and close all OpenCV windows
        cv2.waitKey(0)
        cv2.destroyAllWindows()

        # Release the cameras
        cap1.release()
        cap2.release()

    else:
        raise ValueError(f"Invalid value for n_cams: {n_cams}. Expected 1 or 2.")


def live_preview(n_cams=1, brightness=50, contrast=60, saturation=70, hue=0, temperature=4600, exposure=1000, pan = 0):
    """
    Displays live previews from USB cameras.

    Parameters:
    - n_cams (int): Number of cameras to use. Must be 1 or 2.
    - brightness (int): Brightness level (0-100). Defaults to 50.
    - contrast (int): Contrast level (0-100). Defaults to 60.
    - saturation (int): Saturation level (0-100). Defaults to 70.
    - hue (int): Hue level (0-100). Defaults to 0.
    - temperature (int): White balance temperature (Kelvin). Defaults to 4600.
    - exposure (int): Exposure time in milliseconds. Defaults to 1000.
    - pan (int) : min=-57600 max=57600 step=3600 default=0 value=0.

    Raises:
    - ValueError: If n_cams is not 1 or 2.

    Notes:
    - Uses OpenCV (cv2) to interact with the cameras and display the live previews.
    - Requires v4l-utils package installed to function properly.
    """

    if n_cams == 1:
        cap_index1 = get_cam_index(1)
        # Open a connection to the camera
        cap = cv2.VideoCapture(cap_index1)

        # Check if camera opened successfully
        if not cap.isOpened():
            print("Error: Could not open camera.")
            return

        # Set camera properties
        cap.set(cv2.CAP_PROP_BRIGHTNESS, brightness)
        cap.set(cv2.CAP_PROP_CONTRAST, contrast)
        cap.set(cv2.CAP_PROP_SATURATION, saturation)
        cap.set(cv2.CAP_PROP_HUE, hue)
        cap.set(cv2.CAP_PROP_TEMPERATURE, temperature)
        cap.set(cv2.CAP_PROP_EXPOSURE, exposure)
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        cap.set(cv2.CAP_PROP_PAN, pan)

        while True:
            # Capture frame-by-frame
            ret, frame = cap.read()

            # Check if the frame was captured successfully
            if not ret:
                print("Error: Could not read frame.")
                break

            # Display the captured frame
            cv2.imshow('Live Camera Preview', frame)

            # Press 'q' on the keyboard to exit the live preview
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        # Release the camera and close all OpenCV windows
        cap.release()
        cv2.destroyAllWindows()

    elif n_cams == 2:
        cap_index_1, cap_index_2 = get_cam_index(2)
        
        # Open connections to both cameras
        cap1 = cv2.VideoCapture(cap_index_1)
        cap2 = cv2.VideoCapture(cap_index_2)

        # Check if cameras opened successfully
        if not cap1.isOpened() or not cap2.isOpened():
            print("Error: Could not open one or more cameras.")
            return

        # Set camera properties for both cameras
        cap1.set(cv2.CAP_PROP_BRIGHTNESS, brightness)
        cap1.set(cv2.CAP_PROP_CONTRAST, contrast)
        cap1.set(cv2.CAP_PROP_SATURATION, saturation)
        cap1.set(cv2.CAP_PROP_HUE, hue)
        cap1.set(cv2.CAP_PROP_TEMPERATURE, temperature)
        cap1.set(cv2.CAP_PROP_EXPOSURE, exposure)
        cap1.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap1.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        cap1.set(cv2.CAP_PROP_PAN, pan)
        
        cap2.set(cv2.CAP_PROP_BRIGHTNESS, brightness)
        cap2.set(cv2.CAP_PROP_CONTRAST, contrast)
        cap2.set(cv2.CAP_PROP_SATURATION, saturation)
        cap2.set(cv2.CAP_PROP_HUE, hue)
        cap2.set(cv2.CAP_PROP_TEMPERATURE, temperature)
        cap2.set(cv2.CAP_PROP_EXPOSURE, exposure)
        cap2.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap2.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        cap2.set(cv2.CAP_PROP_PAN, pan)

        while True:
            # Capture frames from both cameras
            ret1, frame1 = cap1.read()
            ret2, frame2 = cap2.read()

            # Check if frames were captured successfully
            if not ret1 or not ret2:
                print("Error: Could not read frames from one or more cameras.")
                break

            # Display frames from both cameras
            cv2.imshow('Live Camera 1 Preview', frame1)
            cv2.imshow('Live Camera 2 Preview', frame2)

            # Press 'q' on the keyboard to exit the live preview
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        # Release the cameras and close all OpenCV windows
        cap1.release()
        cap2.release()
        cv2.destroyAllWindows()

    else:
        raise ValueError(f"Invalid value for n_cams: {n_cams}. Expected 1 or 2.")


def live_preview_with_capture(n_cams=1, brightness=50, contrast=60, saturation=50, hue=0, temperature=4600, exposure=1000, pan=0, start_index=0):
    """
    Displays live previews from USB cameras and captures images on key press.

    Parameters:
    - n_cams (int): Number of cameras to use. Must be 1 or 2.
    - brightness (int): Brightness level (0-100). Defaults to 50.
    - contrast (int): Contrast level (0-100). Defaults to 60.
    - saturation (int): Saturation level (0-100). Defaults to 50.
    - hue (int): Hue level (0-100). Defaults to 0.
    - temperature (int): White balance temperature (Kelvin). Defaults to 4600.
    - exposure (int): Exposure time in milliseconds. Defaults to 1000.
    - pan (int): min=-57600 max=57600 step=3600 default=0 value=0.
    - start_index (int): the numerical value to start naming and saving the images. 

    Raises:
    - ValueError: If n_cams is not 1 or 2.

    Notes:
    - Displays live previews from connected USB cameras using OpenCV (cv2).
    - Captures an image from the live preview and saves it when the space bar is pressed.
    - Press 'q' to exit the live preview.
    - Requires v4l-utils package installed to function properly.
    """

    # Variable to keep track of how many times spacebar is pressed

    image_counter = start_index

    if n_cams == 1:
        cap_index = get_cam_index(1)
        cap = cv2.VideoCapture(cap_index)

        if not cap.isOpened():
            print("Error: Could not open camera.")
            return

        cap.set(cv2.CAP_PROP_BRIGHTNESS, brightness)
        cap.set(cv2.CAP_PROP_CONTRAST, contrast)
        cap.set(cv2.CAP_PROP_SATURATION, saturation)
        cap.set(cv2.CAP_PROP_HUE, hue)
        cap.set(cv2.CAP_PROP_TEMPERATURE, temperature)
        cap.set(cv2.CAP_PROP_EXPOSURE, exposure)
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 480)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        cap.set(cv2.CAP_PROP_PAN, pan)

        while True:
            ret, frame = cap.read()

            if not ret:
                print("Error: Could not read frame.")
                break

            cv2.imshow('Live Camera Preview', frame)

            key = cv2.waitKey(1) & 0xFF
            if key == ord(' '):  # Press space to capture the image
                image_counter += 1
                file_name = f'../../Figures/Cam_1/captured_image_{image_counter}.jpg'
                cv2.imwrite(file_name, frame)
                print(f"Image captured and saved as '{file_name}'.")
            elif key == ord('q'):  # Press 'q' to quit the live preview
                break

        cap.release()
        cv2.destroyAllWindows()

    elif n_cams == 2:
        cap_index1, cap_index2 = get_cam_index(2)
        cap1 = cv2.VideoCapture(cap_index1)
        cap2 = cv2.VideoCapture(cap_index2)

        if not cap1.isOpened() or not cap2.isOpened():
            print("Error: Could not open one or more cameras.")
            return

        cap1.set(cv2.CAP_PROP_BRIGHTNESS, brightness)
        cap1.set(cv2.CAP_PROP_CONTRAST, contrast)
        cap1.set(cv2.CAP_PROP_SATURATION, saturation)
        cap1.set(cv2.CAP_PROP_HUE, hue)
        cap1.set(cv2.CAP_PROP_TEMPERATURE, temperature)
        cap1.set(cv2.CAP_PROP_EXPOSURE, exposure)
        cap1.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap1.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        cap1.set(cv2.CAP_PROP_PAN, pan)
        
        cap2.set(cv2.CAP_PROP_BRIGHTNESS, brightness)
        cap2.set(cv2.CAP_PROP_CONTRAST, contrast)
        cap2.set(cv2.CAP_PROP_SATURATION, saturation)
        cap2.set(cv2.CAP_PROP_HUE, hue)
        cap2.set(cv2.CAP_PROP_TEMPERATURE, temperature)
        cap2.set(cv2.CAP_PROP_EXPOSURE, exposure)
        cap2.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap2.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        cap2.set(cv2.CAP_PROP_PAN, pan)

        while True:
            ret1, frame1 = cap1.read()
            ret2, frame2 = cap2.read()

            if not ret1 or not ret2:
                print("Error: Could not read frames from one or more cameras.")
                break

            cv2.imshow('Live Camera 1 Preview', frame1)
            cv2.imshow('Live Camera 2 Preview', frame2)

            key = cv2.waitKey(1) & 0xFF
            if key == ord(' '):  # Press space to capture the images
                image_counter += 1
                file_name1 = f'../../Figures/Cam_1/captured_image_{image_counter}.jpg'
                file_name2 = f'../../Figures/Cam_2/captured_image_{image_counter}.jpg'
                cv2.imwrite(file_name1, frame1)
                cv2.imwrite(file_name2, frame2)
                print(f"Images captured and saved as '{file_name1}' and '{file_name2}'.")
            elif key == ord('q'):  # Press 'q' to quit the live preview
                break

        cap1.release()
        cap2.release()
        cv2.destroyAllWindows()

    else:
        raise ValueError(f"Invalid value for n_cams: {n_cams}. Expected 1 or 2.")

### Click an image and save it

Press "q" on your keyboard to stop the code and the preview of the image

In [4]:
if __name__ == "__main__":
    click_an_image(n_cams = 1, brightness = 50, saturation = 50, contrast= 60, hue=0, temperature=4600, exposure=300)

Brightness: 50.0
Contrast: 60.0
Saturation: 50.0
Hue: 0.0
White Balance Temperature: 4600.0
Exposure: 300.0


QObject::moveToThread: Current thread (0x28f3ee0) is not the object's thread (0x2706fa0).
Cannot move to target thread (0x28f3ee0)

QObject::moveToThread: Current thread (0x28f3ee0) is not the object's thread (0x2706fa0).
Cannot move to target thread (0x28f3ee0)

QObject::moveToThread: Current thread (0x28f3ee0) is not the object's thread (0x2706fa0).
Cannot move to target thread (0x28f3ee0)

QObject::moveToThread: Current thread (0x28f3ee0) is not the object's thread (0x2706fa0).
Cannot move to target thread (0x28f3ee0)

QObject::moveToThread: Current thread (0x28f3ee0) is not the object's thread (0x2706fa0).
Cannot move to target thread (0x28f3ee0)

QObject::moveToThread: Current thread (0x28f3ee0) is not the object's thread (0x2706fa0).
Cannot move to target thread (0x28f3ee0)

QObject::moveToThread: Current thread (0x28f3ee0) is not the object's thread (0x2706fa0).
Cannot move to target thread (0x28f3ee0)

QObject::moveToThread: Current thread (0x28f3ee0) is not the object's thread

### Show live preview of the camera

You may use this to check the manual focussing of the camera. Press "q" to 

In [6]:
live_preview(n_cams = 1, brightness = 50, saturation = 50, contrast= 80, hue=0, temperature=4600, exposure=500)

QObject::moveToThread: Current thread (0x2684f30) is not the object's thread (0x271af60).
Cannot move to target thread (0x2684f30)

QObject::moveToThread: Current thread (0x2684f30) is not the object's thread (0x271af60).
Cannot move to target thread (0x2684f30)

QObject::moveToThread: Current thread (0x2684f30) is not the object's thread (0x271af60).
Cannot move to target thread (0x2684f30)

QObject::moveToThread: Current thread (0x2684f30) is not the object's thread (0x271af60).
Cannot move to target thread (0x2684f30)

QObject::moveToThread: Current thread (0x2684f30) is not the object's thread (0x271af60).
Cannot move to target thread (0x2684f30)

QObject::moveToThread: Current thread (0x2684f30) is not the object's thread (0x271af60).
Cannot move to target thread (0x2684f30)

QObject::moveToThread: Current thread (0x2684f30) is not the object's thread (0x271af60).
Cannot move to target thread (0x2684f30)

QObject::moveToThread: Current thread (0x2684f30) is not the object's thread

### Click images (pressing space bar to click image) while live preview is on

You may use it to capture images for capturing images for calibration of camera

In [ ]:
live_preview_with_capture(n_cams = 1, brightness = 50, saturation = 50, contrast= 80, hue=0, temperature=4600, exposure=500)